In [7]:
import os
import pickle
import numpy as np
from fashion_mnist import mnist_reader
from vectorization import *
import os
from skimage import io
import pandas as pd

## Generate and save persistence diagrams

In [14]:
folder = 'Outex-TC-00024/data/images'
images_names = os.listdir(folder)
images_names = list(filter(lambda x : x[0]!='.', images_names))

In [15]:
images_matrixes = np.array(list(map(lambda x : io.imread(folder+'/'+x), images_names)), dtype=float)

In [16]:
path = "Outex-TC-00024/data/000/"
train_names = pd.read_csv(path + "train.txt", sep=" ", usecols=[0]).to_numpy().flatten().tolist()
train_labels = pd.read_csv(path + "train.txt", sep=" ", usecols=[1]).to_numpy().flatten().tolist()
test_names = pd.read_csv(path + "test.txt", sep=" ", usecols=[0]).to_numpy().flatten().tolist()
test_labels = pd.read_csv(path + "test.txt", sep=" ", usecols=[1]).to_numpy().flatten().tolist()

In [17]:
train_indexes = list(map(lambda x : images_names.index(x), train_names))
test_indexes = list(map(lambda x : images_names.index(x), test_names))

In [18]:
images_gudhi = np.array(list(map(lambda x : x.reshape(128*128,1), images_matrixes)))
gudhi_train =  images_gudhi[train_indexes]
gudhi_test = images_gudhi[test_indexes]

gudhi_train_opp =  255 - gudhi_train
gudhi_test_opp = 255 - gudhi_test

In [19]:
path_diag = "Outex-TC-00024/pdiagrams/"

In [20]:
for i in range(1360):
    dgms = GetCubicalComplexPDs(img=gudhi_train[i], img_dim=[128,128])
    np.savetxt(path_diag + "l_d0_"+str(i),dgms[0])
    np.savetxt(path_diag + "l_d1_"+str(i),dgms[1])
    
    dgms = GetCubicalComplexPDs(img=gudhi_train_opp[i], img_dim=[128,128])
    np.savetxt(path_diag + "u_d0_"+str(i),dgms[0])
    np.savetxt(path_diag + "u_d1_"+str(i),dgms[1])

In [21]:
for i in range(1360):
    j = 1360+i
    dgms = GetCubicalComplexPDs(img=gudhi_test[i], img_dim=[128,128])
    np.savetxt(path_diag + "l_d0_"+str(j),dgms[0])
    np.savetxt(path_diag + "l_d1_"+str(j),dgms[1])
    
    dgms = GetCubicalComplexPDs(img=gudhi_test_opp[i], img_dim=[128,128])
    np.savetxt(path_diag + "u_d0_"+str(j),dgms[0])
    np.savetxt(path_diag + "u_d1_"+str(j),dgms[1])

## Load Barcodes

In [8]:
pdiagrams = dict()
path_diag = "Outex-TC-00024/pdiagrams/"

#Barcodes with just one bar are loaded as a 1d-array.
#We force them to be a 2d-array
def safe_load(x):
    pd = np.loadtxt(x)
    if (len(pd.shape)==1) and (pd.shape[0]>0): 
        pd = pd.reshape(1,2)
    return pd

for i in range(2720):
    pdiagrams["pdiag_l_d0_"+str(i)]= safe_load(path_diag + "l_d0_"+str(i))
    pdiagrams["pdiag_l_d1_"+str(i)]= safe_load(path_diag + "l_d1_"+str(i))
    pdiagrams["pdiag_u_d0_"+str(i)]= safe_load(path_diag + "u_d0_"+str(i))
    pdiagrams["pdiag_u_d1_"+str(i)]= safe_load(path_diag + "u_d1_"+str(i))

## Calculate Features

In [9]:
func_list = [GetPersStats,
             GetPersImageFeature,
             GetPersLandscapeFeature,
             GetPersEntropyFeature,
             GetBettiCurveFeature,
             GetCarlssonCoordinatesFeature,
             GetPersSilhouetteFeature,
             GetTopologicalVectorFeature,
             GetAtolFeature,
             GetComplexPolynomialFeature,
             GetPersLifespanFeature,
             GetPersTropicalCoordinatesFeature
            ]

In [10]:
features_l_d0 = dict()
features_l_d1 = dict()
features_u_d0 = dict()
features_u_d1 = dict()

for func in func_list:
    print(func.__name__)
    for i in range(2720):
        features_l_d0[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_l_d0_"+str(i)])
        features_l_d1[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_l_d1_"+str(i)])
        features_u_d0[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_u_d0_"+str(i)])
        features_u_d1[(func.__name__)+'_'+str(i)]=func(pdiagrams["pdiag_u_d1_"+str(i)])

GetPersStats
GetPersImageFeature
GetPersLandscapeFeature
GetPersEntropyFeature
GetBettiCurveFeature
GetCarlssonCoordinatesFeature
GetPersSilhouetteFeature
GetTopologicalVectorFeature


/home/manu/review/review/lib/python3.8/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


GetAtolFeature
GetComplexPolynomialFeature


/home/manu/review/review/lib/python3.8/site-packages/gudhi/representations/vector_methods.py:579: RuntimeWarning: overflow encountered in cdouble_scalars
  coeff[j] += ((-1) * roots[i-1] * coeff[j+1])
/home/manu/review/review/lib/python3.8/site-packages/gudhi/representations/vector_methods.py:579: RuntimeWarning: invalid value encountered in cdouble_scalars
  coeff[j] += ((-1) * roots[i-1] * coeff[j+1])


GetPersLifespanFeature
GetPersTropicalCoordinatesFeature


In [11]:
func = GetComplexPolynomialFeature
complex_coeff = []

for i in range(2720):
    complex_coeff.append(features_l_d0[(func.__name__)+'_'+str(i)])
for i in range(2720):
    complex_coeff.append(features_l_d1[(func.__name__)+'_'+str(i)])
for i in range(2720):
    complex_coeff.append(features_u_d0[(func.__name__)+'_'+str(i)])
for i in range(2720):
    complex_coeff.append(features_u_d1[(func.__name__)+'_'+str(i)])

complex_coeff = float64to32(complex_coeff)

for i in range(2720):
    features_l_d0[(func.__name__)+'_'+str(i)]=complex_coeff[i]
    features_l_d1[(func.__name__)+'_'+str(i)]=complex_coeff[2070+i]  
    features_u_d0[(func.__name__)+'_'+str(i)]=complex_coeff[2*2070+i]   
    features_u_d1[(func.__name__)+'_'+str(i)]=complex_coeff[3*2070+i]


In [12]:
path_feat = "Outex-TC-00024/features/"

with open(path_feat + 'l_d0.pkl', 'wb') as f:
  pickle.dump(features_l_d0, f)
with open(path_feat+'l_d1.pkl', 'wb') as f:
  pickle.dump(features_l_d1, f)
with open(path_feat + 'u_d0.pkl', 'wb') as f:
  pickle.dump(features_u_d0, f)
with open(path_feat + 'u_d1.pkl', 'wb') as f:
  pickle.dump(features_u_d1, f)